### Thought process
First, count domains and create a table that has nrow corresponding to dom. count. As the string is very systematic, with a regional expression I can separate the domains with all their information given.  
The good thing here is that the sequence of domains is clear from the string.  
Most important is to know which domain is paired with which. As a human, when I find a bracket in the string, I look for its counterpart. This will have to be done in the second step and put the pairing info into a dictionary(?).

In [1]:
import numpy as np
import re

In [2]:
string = '@90 a( B + ) @45 e* f1'
re.findall('@?[0-9]* ?[a-z|A-Z|0-9|\*]+[ \(]?\+?|\)', string)

['@90 a(', ' B +', ')', '@45 e* ', 'f1']

In [3]:
#error handling step 1
def check_string_valid(string):
    pass

In [11]:
def comprehend_string(string):
    #separate domains/count
    dom_raw_list = re.findall('@?-?[0-9]* ?[a-z|A-Z|0-9|\*]+[ \(]? ?\+?|\) ?\+?', string)
    dom_count = len(dom_raw_list)
        
    #create table - cols: angle, length, dom type, paired info, if there is a strand break after    
    struct_info_array = np.zeros((dom_count, 4))
    
    #pairedness dictionary
    
    #name list of list 
    #name_list = [i.lstrip(' ').rstrip(' (+') for i in dom_list]
    
    return dom_raw_list, struct_info_array

In [12]:
comprehend_string("d1S T2 @-135 b( a( t2( + ) ) c*( t1* + ) )")

(['d1S ',
  'T2 ',
  '@-135 b( ',
  'a( ',
  't2( +',
  ') ',
  ') ',
  'c*( ',
  't1* +',
  ') ',
  ')'],
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]))